# Lego Analysis

Author: M. Tosic
Date: 01.2022
This notebook is part of my capstone project for a data science course.

## 1. Business Understanding

### Questions of interest

General Discovery:
* What themes are dominant over the years?
* What other changes in relation to minifigures or pieces counts are evident?
* Are lego sets becomming more and more expensive?


Lego Investing:
* Does the value of sets go up after eol on average?
* What sets do best after eol?
* What are good predictors of sets that will rise in value after eol?
* What sets that are currently being sold are predicted to be a good investment after eol? (e.g. rise in price of 50%)

## 2. Data Understanding

Data being used in this notebook has been downloaded from the following sources:

* https://brickset.com/
* https://rebrickable.com/downloads/

Simplifications:
* No time series data on the price averages available. Assumption: price changes average out over time after eol.
* No data available on unique minifigs in sets making them potentially more popular for minifig collection.




### Import Libraries

In [139]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import plotly.express as px
#pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_rows', 100)
#pd.set_option('display.max_rows', None)


# import necessary libraries for batch import csv:
import os
import glob

### Import Data

In [23]:
df_sets = pd.read_csv('data/rebrickable-sets.csv')
df_themes = pd.read_csv('data/rebrickable-themes.csv')

In [70]:
def import_multiple_csv_files_2_df (relative_path):
    """ Function uses os and glob packages to import multiple csv files into one dataframe. 
    The current working directory should be the one where this notebook is located.
    INPUT: 
    Relative path to the files e.g. "./data/Kurac*.csv"
    OUTPUT: 
    One dataframe containting all csv files concatenated together over axis = 0.
    """
    path = os.getcwd()
    files = glob.glob(os.path.join(path, relative_path))
    
    print('Glob search with parameters:', relative_path)
   # print('Ingested files:')
    li = []
    for file in files:
        df_temp = pd.read_csv(file, index_col = None, header = 0)
        li.append(df_temp)
        #print(file)
    try:    
        df = pd.concat(li, axis=0, ignore_index=True)
        print('Done.')

    except:
        print('Something went wrong the concatenation of the files, returning None. Is the relative_path correctly set?')
        return(None)
    
    return (df)

In [127]:
df = import_multiple_csv_files_2_df("./data/Brickset*.csv")

Glob search with parameters: ./data/Brickset*.csv
Done.


In [128]:
#Droping unnessecary columns:
df.drop(['Qty owned','UPC','Qty owned new', 
         'Qty owned used', 'EAN','Priority','Wanted', 'Height', 'Depth', 'Weight', 'Width', 
         'Notes','Qty wanted','RRP (CAD)','Flag 1 not used', 'Flag 2 not used', 'Flag 3 not used',
         'Flag 4 not used', 'Flag 5 not used', 'Flag 6 not used','Flag 7 not used', 'Flag 8 not used'], axis=1, inplace=True)

In [129]:
#Adapting columns names to be able to user dot notation and more confortable coding (e.g. price instead of rrp)
df.rename(columns = lambda x : x.replace(' ', '_').replace('(','').replace(')','').lower().strip(), inplace = True)
df.rename(columns={'set':'set_name','rrp_usd': 'price', 'value_new_usd': 'value_new', 'value_used_usd':'value_used'});

In [130]:
#checking types per column
df.dtypes

number             object
theme              object
subtheme           object
year                int64
set_name           object
minifigs          float64
pieces            float64
rrp_gbp           float64
rrp_usd           float64
rrp_eur           float64
value_new_usd     float64
value_used_usd    float64
launch_date        object
exit_date          object
dtype: object

In [131]:
#Parse dates
#df['launch_date'] = pd.to_datetime(df['launch_date'])
#df['exit_date'] = pd.to_datetime(df['exit_date'])

In [132]:
#Exploring content
df.head()

,number,theme,subtheme,year,set_name,minifigs,pieces,rrp_gbp,rrp_usd,rrp_eur,value_new_usd,value_used_usd,launch_date,exit_date
0,10278-1,Creator Expert,Modular Buildings Collection,2021,Police Station,5.0,2923.0,159.99,199.99,179.99,175.03,166.22,01/01/2021,31/12/2024
1,10279-1,Creator Expert,Vehicles,2021,Volkswagen T2 Camper Van,NaN,2207.0,139.99,179.99,159.99,177.20,NaN,01/08/2021,31/12/2022
2,10280-1,Creator Expert,Botanical Collection,2021,Flower Bouquet,NaN,756.0,44.99,49.99,49.99,54.81,45.00,01/01/2021,31/12/2024
3,10281-1,Creator Expert,Botanical Collection,2021,Bonsai Tree,NaN,878.0,44.99,49.99,49.99,51.62,41.73,01/01/2021,31/12/2024
4,10282-1,Creator Expert,Adidas,2021,Adidas Originals Superstar,NaN,731.0,79.99,79.99,89.99,88.45,NaN,01/07/2021,31/12/2023


In [133]:
df.shape

(13894, 14)

In [134]:
df.describe()

,year,minifigs,pieces,rrp_gbp,rrp_usd,rrp_eur,value_new_usd,value_used_usd
count,13894.000000,6094.000000,10318.000000,8178.00000,9208.000000,3919.000000,9146.000000,7469.000000
mean,2012.600043,2.745816,251.683757,26.58214,30.681724,38.636024,75.608434,41.252162
std,6.281093,2.866161,499.906292,39.73432,45.823363,56.598429,213.586669,77.441340
min,2000.000000,1.000000,0.000000,0.00000,0.000000,0.010000,0.100000,0.250000
25%,2008.000000,1.000000,26.000000,5.99000,7.000000,9.990000,10.370000,6.390000
50%,2014.000000,2.000000,84.000000,14.99000,15.000000,19.990000,27.825000,15.970000
75%,2018.000000,3.000000,276.000000,29.99000,35.000000,44.960000,71.162500,42.640000
max,2022.000000,33.000000,11695.000000,699.99000,799.990000,799.990000,9000.000000,1391.390000


In [135]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13894 entries, 0 to 13893
Data columns (total 14 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   number          13894 non-null  object 
 1   theme           13894 non-null  object 
 2   subtheme        11637 non-null  object 
 3   year            13894 non-null  int64  
 4   set_name        13894 non-null  object 
 5   minifigs        6094 non-null   float64
 6   pieces          10318 non-null  float64
 7   rrp_gbp         8178 non-null   float64
 8   rrp_usd         9208 non-null   float64
 9   rrp_eur         3919 non-null   float64
 10  value_new_usd   9146 non-null   float64
 11  value_used_usd  7469 non-null   float64
 12  launch_date     6294 non-null   object 
 13  exit_date       6294 non-null   object 
dtypes: float64(7), int64(1), object(6)
memory usage: 1.5+ MB


In [136]:
print("Percentages of missing values:\n{}".format(df.isnull().sum()/df.shape[0]*100))

Percentages of missing values:
number             0.000000
theme              0.000000
subtheme          16.244422
year               0.000000
set_name           0.000000
minifigs          56.139341
pieces            25.737729
rrp_gbp           41.140060
rrp_usd           33.726789
rrp_eur           71.793580
value_new_usd     34.173024
value_used_usd    46.242983
launch_date       54.699870
exit_date         54.699870
dtype: float64


**Comments:**
* There are NaN values in multiple columns.
* The missing values for minifigs could just be due to the items being lego sets without any minifigures.

**Tasks:**
* A quarter of the items are missing piece counts. This must be investigated since it could indicate the item is not a lego set but some other kind of merchandise from the database.
* Most prices are available in usd, also the value new and used is available in usd. If possible I will try to calculate missing values in usd by the columns of ther currencies.
* Also some dates are missing, I'll take a look at that. Sets from 2022 have probably not yet been release, I will filter them out. For the others I need to look for other explainations.

In [137]:
df[(df.year == 2022) & (df.launch_date.isnull())]

,number,theme,subtheme,year,set_name,minifigs,pieces,rrp_gbp,rrp_usd,rrp_eur,value_new_usd,value_used_usd,launch_date,exit_date
2705,10298-1,Creator Expert,{?},2022,{?},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2706,10299-1,Creator Expert,{?},2022,{?},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2707,10300-1,Creator Expert,{?},2022,{?},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2708,10301-1,Creator Expert,{?},2022,{?},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2709,10302-1,Creator Expert,{?},2022,{?},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2710,10303-1,Creator Expert,{?},2022,{?},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2711,10304-1,Creator Expert,{?},2022,{?},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2712,10305-1,Creator Expert,{?},2022,{?},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2713,10306-1,Creator Expert,{?},2022,{?},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2714,10307-1,Creator Expert,{?},2022,{?},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [140]:
df.shape

(13894, 14)

In [82]:
set(df.theme)

{'4 Juniors',
 'Action Wheelers',
 'Adventurers',
 'Agents',
 'Alpha Team',
 'Aqua Raiders',
 'Architecture',
 'Art',
 'Atlantis',
 'Avatar The Last Airbender',
 'Baby',
 'Basic',
 'Batman',
 'Belville',
 'Ben 10: Alien Force',
 'Bionicle',
 'Books',
 'Boost',
 'Brick Sketches',
 'BrickHeadz',
 'Bricklink',
 'Bricks and More',
 'Bulk Bricks',
 'Cars',
 'Castle',
 'City',
 'Classic',
 'Clikits',
 'Collectable Minifigures',
 'Creator',
 'Creator Expert',
 'DC Comics Super Heroes',
 'DC Super Hero Girls',
 'Dacta',
 'Dimensions',
 'Dino',
 'Dino 2010',
 'Dino Attack',
 'Dinosaurs',
 'Discovery',
 'Disney',
 'Dots',
 'Duplo',
 'Education',
 'Elves',
 'Exo-Force',
 'Explore',
 'FORMA',
 'Factory',
 'Friends',
 'Fusion',
 'Galidor',
 'Games',
 'Gaming',
 'Gear',
 'Ghostbusters',
 'HERO Factory',
 'Harry Potter',
 'Hidden Side',
 'Hobby Set',
 'Ideas',
 'Indiana Jones',
 'Island Xtreme Stunts',
 'Jack Stone',
 'Juniors',
 'Jurassic World',
 'Legends of Chima',
 'Life of George',
 'Make and Cr

In [85]:
df[df.theme == "Books"].head()

,number,theme,subtheme,year,set_name,minifigs,pieces,rrp_gbp,rrp_usd,rrp_eur,value_new_usd,value_used_usd,launch_date,exit_date
829,5006978-1,Books,Dorling Kindersley,2021,Masters' Book of Secrets,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT
883,BLOCKS075-1,Books,Magazines/Blocks,2021,Blocks magazine issue 75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT
884,BLOCKS076-1,Books,Magazines/Blocks,2021,Blocks magazine issue 76,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT
885,BLOCKS077-1,Books,Magazines/Blocks,2021,Blocks magazine issue 77,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT
886,BLOCKS078-1,Books,Magazines/Blocks,2021,Blocks magazine issue 78,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT


I'm only interested in lego sets not books and additional gear. 

So how do I filter those out of the dataset as easy as possible? My assumption is that the sets that contain no pieces are not proper lego sets but rather gear. Also I would like to remove any unreleased items, that would be items with no launch date.

In [91]:
df[df.pieces.isnull()].head(100)

,number,theme,subtheme,year,set_name,minifigs,pieces,rrp_gbp,rrp_usd,rrp_eur,value_new_usd,value_used_usd,launch_date,exit_date
5,10282-2,Creator Expert,Adidas,2021,Adidas Originals Superstar X Footshop 'Bluepri...,NaN,NaN,79.99,79.99,89.99,NaN,NaN,2021-01-07,2023-12-31
58,11956-1,Miscellaneous,NaN,2021,Parts for Super Nature,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT
59,11957-1,Miscellaneous,NaN,2021,Parts for {LEGO Build Book},NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT
137,40458-1,Promotional,LEGO House,2021,LEGO House Chef,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT
162,40489-1,Seasonal,Christmas,2021,Mr. and Mrs. Claus' Living Room,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaT
253,43101-0,Vidiyo,Bandmates Series 1,2021,Bandmates Series 1 {Random box},NaN,NaN,3.99,4.99,4.99,NaN,NaN,2021-01-03,2022-12-31
267,43101-14,Vidiyo,Bandmates Series 1,2021,Bandmates Series 1 - Sealed Box,NaN,NaN,3.99,4.99,4.99,NaN,NaN,2021-01-03,2022-12-31
274,43108-0,Vidiyo,Bandmates Series 2,2021,Bandmates Series 2 {Random box},NaN,NaN,3.99,4.99,4.99,NaN,NaN,2021-01-10,2022-12-31
288,43108-14,Vidiyo,Bandmates Series 2,2021,Bandmates Series 2 - Sealed Box,NaN,NaN,4.49,4.99,4.99,NaN,NaN,2021-01-10,2022-12-31
311,45345-1,Education,SPIKE Essential,2021,SPIKE Essential Set,NaN,NaN,274.99,274.95,304.99,393.22,NaN,NaT,NaT


In [ ]:
["Books"]

In [76]:
def make_bar_chart(df,x_val,y_val,title_string = "",labels_dict = {}):
    '''Function uses plotly.express to create a bar chart and show it in the notebook right-away.
    INPUT
    df: dataframe to be visualizes in bar chart
    x_val: column to be represented on x-axis
    y_val: column to be represented on y-axis
    title_string: the title as a string
    labels_dict: labels can be renamed by entering a dictionary.
    OUTPUT
    None
    '''
    fig = px.bar(df, x_val, y_val, 
            title=title_string, 
            labels=labels_dict)
   # fig.show()
    return fig

In [78]:
make_bar_chart(df,df.year,df.theme)

In [141]:

#Could be needed later 

df.dropna(subset=["minifigs","pieces","rrp_gbp",
                  "rrp_eur","rrp_usd","value_used_usd",
                  "launch_date", "exit_date"], how='all').shape

(12600, 14)

## 3. Prepare Data

## 4. Analysis

## 5. Evaluation